In [ ]:
%pip install comet_ml #libreria a instalar

In [ ]:
import comet_ml
comet_ml.init(project_name="RNA_Densa_Primera") #nombre a dar al experimento

In [ ]:
# Imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import mnist #base de datos
from tensorflow.keras.models import Sequential # checar cada uno de los que faltan
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras import regularizers
from keras.callbacks import ModelCheckpoint # sirve para dar respuesta del proceso de entrenamiento de la red

In [ ]:
experiment = comet_ml.Experiment(  # Para activar las Graficaciones en comet
    auto_histogram_weight_logging=True,
    auto_histogram_gradient_logging=True,
    auto_histogram_activation_logging=True,
    log_code=True, # para subir el codigo a comet para no perder el progreso
)

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/nicglowss/rna-densa-primera/c9d589ea27524f83a5846f7d23795287
COMET INFO:   Parameters:
COMET INFO:     batch_size : 10
COMET INFO:     epochs     : 30
COMET INFO:     loss       : categorical_crossentropy
COMET INFO:     optimizer  : SGD
COMET INFO:   Uploads:
COMET INFO:     environment details : 1
COMET INFO:     filename            : 1
COMET INFO:     installed packages  : 1
COMET INFO:     notebook            : 2
COMET INFO:     os packages         : 1
COMET INFO:     source_code         : 1
COMET INFO: 
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensu

In [ ]:
# pre-procesamiento
dataset=mnist.load_data()# en el dataset se guardan todas las imagenes del mnist con sus etiquetas
(x_train, y_train), (x_test, y_test) = dataset

# se van a guardar en el forato de binario
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# numero de pixeles normalizando el no. de pixeles
x_train /= 255  # x_trainv = x_trainv/255
x_test /= 255

#clasificacion de clases
num_classes=10
y_trainc = keras.utils.to_categorical(y_train, num_classes) #analiza los datos y los acomoda en las distintas clases
y_testc = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
# Parametros
parameters = {
    "batch_size": 10,
    "epochs": 30,
    "optimizer": "SGD",
    "loss": "categorical_crossentropy",
}

experiment.log_parameters(parameters) # diccionario subido a comet

In [ ]:
# Arquitetura de la red
model = Sequential()
# aqui empezamos
model.add(Input(shape=(28,28))) #Flaten no tiene la opcion input_shape por lo tanto se tiene que agregar esta capa
                                #28x28 es el tamaño de la imagen
model.add(Flatten()) #Otra forma de aplanar las imagenes, las forma en una linea
model.add(Dense(784, activation='sigmoid')) # esta es la capa densa de no neuronas de forma vertical (en este caso agarramos el total de pixeles)
                                            # su funcion de activacion es la sigmoide (respuesta de la neurona de 0 a 1)
                                            # Estas dos son las principales

model.add(Dense(10, activation='softmax'))  # son las de salida literal porque estan al final

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 dense_4 (Dense)             (None, 784)               615440    
                                                                 
 dense_5 (Dense)             (None, 10)                7850      
                                                                 
Total params: 623290 (2.38 MB)
Trainable params: 623290 (2.38 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# specify the path where you want to save the model
filepath = "mejor-modelo1.1.hdf5"

# initialize the ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [ ]:
# Inicia el entrenamiento de la red
model.compile(loss=parameters['loss'],optimizer=SGD(learning_rate=0.1,ema_momentum=0.9),metrics=['accuracy'])
model.fit(x_train, y_trainc,
                    batch_size=parameters['batch_size'],
                    epochs=parameters["epochs"],
                    verbose=1,
                    validation_data=(x_test, y_testc),
                    callbacks=[checkpoint])

COMET INFO: Ignoring automatic log_parameter('verbose') because 'keras:verbose' is in COMET_LOGGING_PARAMETERS_IGNORE


Epoch 1/30
5988/6000 [============================>.] - ETA: 0s - loss: 0.4304 - accuracy: 0.8737
Epoch 1: val_loss improved from inf to 0.26707, saving model to mejor-modelo1.1.hdf5
6000/6000 [==============================] - 33s 5ms/step - loss: 0.4300 - accuracy: 0.8738 - val_loss: 0.2671 - val_accuracy: 0.9210
Epoch 2/30
   1/6000 [..............................] - ETA: 53s - loss: 0.7038 - accuracy: 0.8000

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


5995/6000 [============================>.] - ETA: 0s - loss: 0.2467 - accuracy: 0.9276
Epoch 2: val_loss improved from 0.26707 to 0.20705, saving model to mejor-modelo1.1.hdf5
6000/6000 [==============================] - 31s 5ms/step - loss: 0.2466 - accuracy: 0.9277 - val_loss: 0.2070 - val_accuracy: 0.9415
Epoch 3/30
5998/6000 [============================>.] - ETA: 0s - loss: 0.1821 - accuracy: 0.9470
Epoch 3: val_loss improved from 0.20705 to 0.15493, saving model to mejor-modelo1.1.hdf5
6000/6000 [==============================] - 29s 5ms/step - loss: 0.1822 - accuracy: 0.9470 - val_loss: 0.1549 - val_accuracy: 0.9544
Epoch 4/30
5993/6000 [============================>.] - ETA: 0s - loss: 0.1425 - accuracy: 0.9590
Epoch 4: val_loss improved from 0.15493 to 0.12789, saving model to mejor-modelo1.1.hdf5
6000/6000 [==============================] - 31s 5ms/step - loss: 0.1425 - accuracy: 0.9590 - val_loss: 0.1279 - val_accuracy: 0.9626
Epoch 5/30
5995/6000 [==========================

In [ ]:
experiment.log_model("MNIST1", "mejor-modelo1.1.hdf5")

{'web': 'https://www.comet.com/api/asset/download?assetId=4c2a73bc95b2487a9793651a3319a235&experimentKey=016f804e079b4a858eab944236d7dc2f',
 'api': 'https://www.comet.com/api/rest/v2/experiment/asset/get-asset?assetId=4c2a73bc95b2487a9793651a3319a235&experimentKey=016f804e079b4a858eab944236d7dc2f',
 'assetId': '4c2a73bc95b2487a9793651a3319a235'}

In [ ]:
experiment.end()

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/nicglowss/rna-densa-primera/016f804e079b4a858eab944236d7dc2f
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     accuracy [30]                  : (0.8737833499908447, 0.9991333484649658)
COMET INFO:     batch_accuracy [18000]         : (0.0, 1.0)
COMET INFO:     batch_loss [18000]             : (0.00019927509129047394, 5.7957329750061035)
COMET INFO:     epoch_duration [30]            : (26.240303262999873, 37.187145596000164)
COMET INFO:     loss [30]                      : (0.008802610449492931, 0.43003717064857483)
COMET INFO:     val_accuracy [30]              : (0.9210000038146973, 0.9814000129699707)
COMET INFO:     